# Configure a Small Core Network
This notebook shows you the code required to bring up a small core network with predefined configurations, using the Pyvxr API. The configuration are derived from [Compass Core Configs](https://github.com/ios-xr/design/tree/master/Core).

These functions are then used in the small_core.py library which we use to bring this network up again in other notebooks.

![Small Core Diagram](SmallCore.png)

***API Docs: [Read the PYVXR docs](http://pyvxr.cisco.com/ciscovxr/index.html)***

The following sets up our python enviroment and the emulator object.

In [16]:
from pathlib import Path
from pyvxr.vxr import Vxr
import sys
import os
import logging
import shutil
import getpass
import uuid
import string
import random
print(sys.version)
logging.basicConfig(level=logging.INFO)
sys.path.append("../../../")
from image_version import *

# Setting up scratch space for the simulation
sim_dir = '/nobackup/' + getpass.getuser() + '/pyvxr/' + ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))
sim = Vxr()
sim.no_image_copy=True

INFO:pyvxr.vxr:v1.1.0 2021-04-23 05:27 output_dir:vxr.out
INFO:pyvxr.vxr:b10e8f6b8c14:/home/vxr/notebooks/Getting-Started/Setting-Up-Basic-Network/Small-Core


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


This defines and starts up a small two router topology for our lab and a Linux server to use on the same LAN, with SSH to all of the routers.
Takes a minute or five to start. 

In [17]:
# Nice little python trick to peek at the API of a object.
dir(sim)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allocate_slurm',
 '_analyze_logs',
 '_check_sim_config_yaml_file',
 '_clean_sim',
 '_connect_distributed_hubs',
 '_create_fresh_dir',
 '_create_port_yaml_file',
 '_create_vxrsim_cfg_file',
 '_download_dot_server_files',
 '_fix_vxr_release_string',
 '_get_pyvxr_flag',
 '_get_vxr_ports_file',
 '_get_vxr_ports_files',
 '_get_vxr_rel',
 '_get_vxr_ver',
 '_get_vxr_version',
 '_logs_from_host',
 '_parse_routers',
 '_portVectorFileName',
 '_post_sim_launch',
 '_restart_cards',
 '_restart_common',
 '_restart_sim',
 '_run_sim_check',
 '_send_email',
 '_setup_local_ssh_tunnels',
 '_setup_ssh_tunnels',
 '_start_common'

Configurations for bringing up routers with some external SSH reachability.

In [18]:
ssh_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh
"""

In [19]:
p1_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh

hostname P1
line console
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
line default
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
 
ipv4 unnumbered mpls traffic-eng Loopback0
key chain ISIS-KEY
 key 1
  accept-lifetime 00:00:00 january 01 2018 infinite
  key-string password 121A0C041104
  send-lifetime 00:00:00 january 01 2018 infinite
  cryptographic-algorithm HMAC-MD5
 !
!
interface Bundle-Ether57
 description Connected_to_P3
 bfd mode ietf
 bfd address-family ipv4 timers start 180
 bfd address-family ipv4 multiplier 3
 bfd address-family ipv4 destination 10.5.7.2
 bfd address-family ipv4 fast-detect
 bfd address-family ipv4 minimum-interval 50
 mtu 9216
 ipv4 address 10.5.7.1 255.255.255.0
 ipv4 unreachables disable
 bundle minimum-active links 1
 load-interval 30
 damp
!
interface Loopback0
 ipv4 address 10.0.0.5 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 bundle id 57 mode on
 no shutdown
!
interface FourHundredGigE0/0/0/1
 bundle id 57 mode on
 no shutdown
 !
 router isis CORE
 net 49.0001.0100.0000.0005.00
 nsr
 nsf cisco
 log adjacency changes
 lsp-gen-interval maximum-wait 5000 initial-wait 50 secondary-wait 200
 lsp-refresh-interval 65000
 max-lsp-lifetime 65535
 lsp-password keychain ISIS-KEY
 lsp-password keychain ISIS-KEY level 2
 address-family ipv4 unicast
  metric-style wide
  spf-interval maximum-wait 5000 initial-wait 50 secondary-wait 200
  spf prefix-priority critical tag 5000
  spf prefix-priority high tag 1000
 !
!
mpls oam
!
mpls ldp
 router-id 10.0.0.5
 interface Bundle-Ether57
 !
!
multicast-routing
 address-family ipv4
  interface all enable
 !
 """

In [20]:
p3_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh

hostname P3
!
ipv4 unnumbered mpls traffic-eng Loopback0
key chain ISIS-KEY
 key 1
  accept-lifetime 00:00:00 january 01 2018 infinite
  key-string password 110A1016141D
  send-lifetime 00:00:00 january 01 2018 infinite
  cryptographic-algorithm HMAC-MD5
 !
!
interface Bundle-Ether57
 description Connected_to_P1
 bfd mode ietf
 bfd address-family ipv4 timers start 180
 bfd address-family ipv4 multiplier 3
 bfd address-family ipv4 destination 10.5.7.1
 bfd address-family ipv4 fast-detect
 bfd address-family ipv4 minimum-interval 50
 mtu 9216
 ipv4 address 10.5.7.2 255.255.255.0
 ipv4 unreachables disable
 bundle minimum-active links 1
 load-interval 30
 dampening
!
interface Loopback0
 ipv4 address 10.0.0.7 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 bundle id 57 mode on
 no shutdown
!
interface FourHundredGigE0/0/0/1
 bundle id 57 mode on
 no shutdown
!
router isis CORE
 is-type level-2-only
 net 49.0001.0100.0000.0007.00
 nsr
 nsf cisco
 log adjacency changes
 lsp-gen-interval maximum-wait 5000 initial-wait 50 secondary-wait 200
 lsp-refresh-interval 65000
 max-lsp-lifetime 65535
 lsp-password keychain ISIS-KEY
 lsp-password keychain ISIS-KEY level 2
 address-family ipv4 unicast
  metric-style wide
  spf-interval maximum-wait 5000 initial-wait 50 secondary-wait 200
  spf prefix-priority critical tag 5000
  spf prefix-priority high tag 1000
 !
!
mpls oam
!
mpls ldp
 router-id 10.0.0.7
 interface Bundle-Ether57
 !
!
multicast-routing
 address-family ipv4
  interface all enable
 !
!
"""

In [21]:
cfg = { 'simulation':
         {'skip_auto_bringup': False, 
          'sim_dir': sim_dir, 
          'sim_host': 'localhost',
          'sim_rel': '/opt/cisco/vxr2/latest',
          'pyvxr_flags': {'port_file_timeout': 1200 }
         },
        'devices':
        {'rp1': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'linecard_types': ['8201-sys'], 
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1'],
                'xr_config' : p1_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
                },
         'rp3': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'linecard_types': ['8201-sys'], 
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1'],
                'xr_config' : p3_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
                }
        },
       'connections':
           {'hubs':
               {'hub570':['rp1.FourH0/0/0/0', 'rp3.FourH0/0/0/0'],
               'hub571':['rp1.FourH0/0/0/1', 'rp3.FourH0/0/0/1']},
           },
      }

In [22]:
sim.clean()
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")
try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/7wpebt6art sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:rp1, rp3
INFO:pyvxr.console:rp3:wait for XR login prompt (console output captured in vxr.out/logs/console.rp3.log)
INFO:pyvxr.console:rp1:wait for XR login prompt (console output captured in vxr.out/logs/console.rp1.log)
INFO:pyvxr.console:rp3:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:rp3:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:rp3:login successful
INFO:pyvxr.bringup:rp3:wait for IOS XR RUN state
INFO:pyvxr.bringup:rp3:run state RPs:1 (expected:1) LCs:0 (expected:0)
INFO:pyvxr.bringup:rp3:all nodes reached IOS XR RUN state
INFO:pyvxr.console:rp3:applying initial XR config (terminal width, etc)


Sim status:  {'localhost': 'running'}


In [23]:
# Defining two little helpers to make accessing the routers easy.

def get_telnet_cmd(sim, router):
    """Get a telnet command to a router in a VXR simulation.
    Keyword arguments:
    sim -- an instance of the Vxr object
    router -- the router name in the simulation
    """
    console_ports = sim.ports()
    return "telnet " + str(console_ports[router]['HostAgent']) + ' ' + str(console_ports[router]['serial0'])

def get_ssh_cmd(sim, device, is_server=False):
    """Get a telnet command to a router in a VXR simulation
    Keyword arguments:
    sim -- an instance of the Vxr object
    router -- the router name in the simulation
    """
    console_ports = sim.ports()
    if (is_server):
        return "ssh root@" + str(console_ports[device]['HostAgent']) + ' -p' + str(console_ports[device]['xr_redir22'])
    else:
        return "ssh cisco@" + str(console_ports[device]['HostAgent']) + ' -p' + str(console_ports[device]['xr_redir22'])

In [24]:
print('Consoles can be reached by:')
print(get_telnet_cmd(sim, 'rp1'), '\n', get_telnet_cmd(sim, 'rp3'))
print('or better:')
print(get_ssh_cmd(sim, 'rp1'), '\n', get_ssh_cmd(sim, 'rp3'))
print('The password is cisco123')

Consoles can be reached by:
telnet 172.17.0.2 34263 
 telnet 172.17.0.2 42898
or better:
ssh cisco@172.17.0.2 -p63296 
 ssh cisco@172.17.0.2 -p63084
The password is cisco123


The sim ports is a json structure, that contains the information. You can either open your terminal and directly access the consoles of the routers or you can just continue playing the next steps and access the router consoles from this notebook.

In [25]:
import telnetlib
console_ports = sim.ports()
loginp1 = telnetlib.Telnet(str(console_ports['rp1']['HostAgent']) , str(console_ports['rp1']['serial0']))
loginp3 = telnetlib.Telnet(str(console_ports['rp3']['HostAgent']) , str(console_ports['rp3']['serial0']))

You can now try look at the interfaces and ping across it.

In [26]:
loginp1.write(('''
show bundle bundle-ether 57
show ip int br
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:P1#show bundle bundle-ether 57
Fri Apr 23 05:32:20.694 UTC

Bundle-Ether57
  Status:                                    Down
  Local links <active/standby/configured>:   0 / 0 / 2
  Local bandwidth <effective/available>:     0 (0) kbps
  MAC address (source):                      78a0.30c2.cd04 (Chassis pool)
  Inter-chassis link:                        No
  Minimum active links / bandwidth:          1 / 1 kbps
  Maximum active links:                      64
  Wait while timer:                          2000 ms
  Load balancing:                            
    Link order signaling:                    Not configured
    Hash type:                               Default
    Locality threshold:                      None
  LACP:                                      Not operational
    Flap suppression timer:                  Off
    Cisco extensions:                        Disabled
    Non-revertive:                           Disabled
  mLACP:                                  

Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [27]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/7wpebt6art)
INFO:pyvxr.sim:Stopping previous simulation (if any)


We have other capabilties in the emulator, so we have seperate Notebooks on Telemetry, Packet simulation tests and forwarding limit simptoms,

In [28]:
sim.clean()

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/7wpebt6art)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


> Clean up the sim scratch-space - delete the sim directory 

In [29]:
shutil.rmtree(sim_dir)